# Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [0]:
##### Your Code Here #####
# generic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
#from google.colab import files
#files.upload()

In [0]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head(20)
df.shape

# Customer ID is unique across so dropping it
df.drop(columns='customerID', inplace=True)
df['TotalCharges'] = df['TotalCharges'].apply(lambda x: 0.0 if len(x) < 2 else float(x))

# Use the values to get np.arrary instead of passing the dataframe
X = df.drop(columns='Churn').values
y = df['Churn'].values

In [0]:
for column in df.columns:
  print(column, df[column].dtype, df[column].nunique())

gender object 2
SeniorCitizen int64 2
Partner object 2
Dependents object 2
tenure int64 73
PhoneService object 2
MultipleLines object 3
InternetService object 3
OnlineSecurity object 3
OnlineBackup object 3
DeviceProtection object 3
TechSupport object 3
StreamingTV object 3
StreamingMovies object 3
Contract object 3
PaperlessBilling object 2
PaymentMethod object 4
MonthlyCharges float64 1585
TotalCharges float64 6531
Churn object 2


In [0]:
# Use category encoder to update object columns
!pip install category_encoders

In [0]:
# Imports for pipeline
from sklearn.pipeline import make_pipeline

import category_encoders as ce
from sklearn.preprocessing import RobustScaler

from sklearn.model_selection import GridSearchCV

# Keras imports
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=44, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

# Create pipeline
pipeline = make_pipeline(\
                         ce.BinaryEncoder(),
                         RobustScaler(), 
                         KerasClassifier(build_fn=create_model, verbose=1))

# Model validation.
param_grid = {
}

gridsearch = GridSearchCV(pipeline, param_grid=param_grid, cv=3, 
                         scoring='accuracy', verbose=10)

gridsearch.fit(X, y)

# Interpret the results.

# Best cross validation score
print('Cross Validation Score:', gridsearch.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch.best_params_)

Using TensorFlow backend.


Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV]  ................................................................
Instructions for updating:
Colocations handled automatically by placer.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Instructions for updating:
Use tf.cast instead.
Epoch 1/1
4695/4695 [==============================] - 0s 9us/step
[CV] ....................... , score=0.7904599659284497, total=   0.9s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s


Epoch 1/1
4695/4695 [==============================] - 0s 10us/step
[CV] ....................... , score=0.7781090289608177, total=   1.0s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.3s remaining:    0.0s


Epoch 1/1
4696/4696 [==============================] - 0s 10us/step
[CV] ....................... , score=0.7869620792501065, total=   1.0s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    3.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    3.4s finished


Epoch 1/1
7043/7043 [==============================] - 0s 56us/step - loss: 0.4952 - acc: 0.7470
Cross Validation Score: 0.7851767712622462
Best Parameters: {}


Without any hyper parameter tuning:

Cross Validation Score: 0.7755217946897629

 Hyper parameter tuning:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer

In [0]:
from keras.optimizers import Adam
from keras.layers import advanced_activations

# fix random seed for reproducibility
np.random.seed(42)

# Function to create model, required for KerasClassifier
def create_model(activation_hidden='relu', learning_rate=0.001, rate=0.1,
                units=12):
                 #optimizer='adam'):
  # create model
  model = Sequential()
  model.add(Dense(units, input_dim=44, activation=activation_hidden))
  model.add(Dropout(rate))
  model.add(Dense(1, activation='sigmoid'))
  model.add(Dropout(rate))
  # Compile model
  optimizer = Adam(lr=learning_rate)
  model.compile(loss='binary_crossentropy', optimizer=optimizer, \
                metrics=['accuracy'])
  return model

# Create pipeline
pipeline = make_pipeline(\
                         ce.BinaryEncoder(),
                         RobustScaler(), 
                         KerasClassifier(build_fn=create_model, verbose=1))

# Model validation.
param_grid = {
    'kerasclassifier__batch_size': [80],
    'kerasclassifier__epochs': [50],
    #'kerasclassifier__optimizer': ['sgd', 'rmsprop', 'adagrad', 'adadelta',\
    #'adam', 'adamax', 'nadam'],
    'kerasclassifier__learning_rate': [0.001],
    'kerasclassifier__activation_hidden': ['exponential'],
    'kerasclassifier__rate': [0.1],
    'kerasclassifier__units': [64]
}

gridsearch = GridSearchCV(pipeline, param_grid=param_grid, cv=3, 
                         scoring='accuracy', verbose=10)

gridsearch.fit(X, y)

# Interpret the results.

# Best cross validation score
print('Cross Validation Score:', gridsearch.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] kerasclassifier__activation_hidden=exponential, kerasclassifier__batch_size=80, kerasclassifier__epochs=50, kerasclassifier__learning_rate=0.001, kerasclassifier__rate=0.1, kerasclassifier__units=64 
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/50
4695/4695 [==============================] - 0s 71us/step - loss: 0.9200 - acc: 0.7542
Epoch 2/50
4695/4695 [==============================] - 0s 16us/step - loss: 0.8734 - acc: 0.7742
Epoch 3/50
4695/4695 [==============================] - 0s 16us/step - loss: 0.7690 - acc: 0.7770
Epoch 4/50
4695/4695 [==============================] - 0s 15us/step - loss: 0.8802 - acc: 0.7734
Epoch 5/50
4695/4695 [==============================] - 0s 17us/step - loss: 0.8265 - acc: 0.7804
Epoch 6/50
4695/4695 [==============================] - 0s 15us/step - loss: 0.8270 - acc: 0.7796
Epoch 7/50
4695/4695 [==============================] - 0s 15us/step - loss: 0.7593 - acc: 0.7853
Epoch 8/50
4695/4695 [==============================] - 0s 16us/step - loss: 0.8129 - acc: 0.7804
Epoch 9/50
4695/4695 [==============================] - 0s 16us/step - loss: 0.8811 - acc: 0.7855
Epoch 10/50
4695/4695 [==============================] - 0s 16us/step - loss: 0.8515 - acc: 0.7853
Epoch 11/50
4695/46

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.8s remaining:    0.0s


Epoch 1/50
4695/4695 [==============================] - 0s 79us/step - loss: 1.2014 - acc: 0.6106
Epoch 2/50
4695/4695 [==============================] - 0s 15us/step - loss: 0.9895 - acc: 0.7308
Epoch 3/50
4695/4695 [==============================] - 0s 15us/step - loss: 0.8654 - acc: 0.7485
Epoch 4/50
4695/4695 [==============================] - 0s 15us/step - loss: 0.8421 - acc: 0.7636
Epoch 5/50
4695/4695 [==============================] - 0s 15us/step - loss: 0.8386 - acc: 0.7693
Epoch 6/50
4695/4695 [==============================] - 0s 15us/step - loss: 0.8912 - acc: 0.7704
Epoch 7/50
4695/4695 [==============================] - 0s 15us/step - loss: 0.8470 - acc: 0.7706
Epoch 8/50
4695/4695 [==============================] - 0s 15us/step - loss: 0.8453 - acc: 0.7749
Epoch 9/50
4695/4695 [==============================] - 0s 16us/step - loss: 0.8257 - acc: 0.7768
Epoch 10/50
4695/4695 [==============================] - 0s 15us/step - loss: 0.7799 - acc: 0.7836
Epoch 11/50
4695/46

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    9.8s remaining:    0.0s


Epoch 1/50
4696/4696 [==============================] - 0s 87us/step - loss: 0.8652 - acc: 0.7287
Epoch 2/50
4696/4696 [==============================] - 0s 15us/step - loss: 0.8108 - acc: 0.7860
Epoch 3/50
4696/4696 [==============================] - 0s 15us/step - loss: 0.8806 - acc: 0.7807
Epoch 4/50
4696/4696 [==============================] - 0s 15us/step - loss: 0.8371 - acc: 0.7828
Epoch 5/50
4696/4696 [==============================] - 0s 15us/step - loss: 0.8187 - acc: 0.7847
Epoch 6/50
4696/4696 [==============================] - 0s 14us/step - loss: 0.8660 - acc: 0.7881
Epoch 7/50
4696/4696 [==============================] - 0s 14us/step - loss: 0.8060 - acc: 0.7875
Epoch 8/50
4696/4696 [==============================] - 0s 15us/step - loss: 0.8093 - acc: 0.7939
Epoch 9/50
4696/4696 [==============================] - 0s 15us/step - loss: 0.7923 - acc: 0.7917
Epoch 10/50
4696/4696 [==============================] - 0s 15us/step - loss: 0.7801 - acc: 0.7922
Epoch 11/50
4696/46

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   14.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   14.7s finished


Epoch 1/50
7043/7043 [==============================] - 0s 69us/step - loss: 0.8878 - acc: 0.7527
Epoch 2/50
7043/7043 [==============================] - 0s 15us/step - loss: 0.7837 - acc: 0.7767
Epoch 3/50
7043/7043 [==============================] - 0s 15us/step - loss: 0.7796 - acc: 0.7873
Epoch 4/50
7043/7043 [==============================] - 0s 14us/step - loss: 0.8540 - acc: 0.7849
Epoch 5/50
7043/7043 [==============================] - 0s 15us/step - loss: 0.8362 - acc: 0.7865
Epoch 6/50
7043/7043 [==============================] - 0s 15us/step - loss: 0.8420 - acc: 0.7850
Epoch 7/50
7043/7043 [==============================] - 0s 14us/step - loss: 0.7742 - acc: 0.7906
Epoch 8/50
7043/7043 [==============================] - 0s 16us/step - loss: 0.7887 - acc: 0.7863
Epoch 9/50
7043/7043 [==============================] - 0s 16us/step - loss: 0.7425 - acc: 0.7940
Epoch 10/50
7043/7043 [==============================] - 0s 14us/step - loss: 0.8407 - acc: 0.7892
Epoch 11/50
7043/70

1:
param_grid = {

    'kerasclassifier__batch_size': [20, 50, 80, 100, 200],
    'kerasclassifier__epochs': [20],
}

Cross Validation Score: 0.8046287093568082

Best Parameters: {'kerasclassifier__batch_size': 80, 'kerasclassifier__epochs': 20}

2: param_grid = {

    'kerasclassifier__batch_size': [80],
    'kerasclassifier__epochs': [20],
    'kerasclassifier__optimizer': ['sgd', 'rmsprop', 'adagrad', 'adadelta', 'adam', 'adamax', 'nadam']
}


Cross Validation Score: 0.8054806190543803

Best Parameters: {'kerasclassifier__batch_size': 80, 'kerasclassifier__epochs': 20, 'kerasclassifier__optimizer': 'adam'}

3: param_grid = {

    'kerasclassifier__batch_size': [80],
    'kerasclassifier__epochs': [20],
    'kerasclassifier__learning_rate': [0.001, 0.01, 0.1, 0.2, 0.5],
}

Cross Validation Score: 0.8007951157177339

Best Parameters: {'kerasclassifier__batch_size': 80, 'kerasclassifier__epochs': 20, 'kerasclassifier__learning_rate': 0.001}

4: param_grid = {

    'kerasclassifier__batch_size': [80],
    'kerasclassifier__epochs': [20],
    'kerasclassifier__learning_rate': [0.001, 0.002, 0.005, 0.01],
}


Cross Validation Score: 0.8009371006673293

Best Parameters: {'kerasclassifier__batch_size': 80, 'kerasclassifier__epochs': 20, 'kerasclassifier__learning_rate': 0.001}

5: param_grid = {

    'kerasclassifier__batch_size': [80],
    'kerasclassifier__epochs': [20],
    'kerasclassifier__learning_rate': [0.001],
    'kerasclassifier__activation_hidden': ['linear', 'exponential', 
                                           'hard_sigmoid', 'sigmoid', 'tanh',
                                           'relu', 'softsign', 'softplus', 
                                           'selu', 'elu', 'softmax'],
}

Cross Validation Score: 0.8039187846088315

Best Parameters: {'kerasclassifier__activation_hidden': 'exponential', 'kerasclassifier__batch_size': 80, 'kerasclassifier__epochs': 20, 'kerasclassifier__learning_rate': 0.001}


6: param_grid = {

    'kerasclassifier__batch_size': [80],
    'kerasclassifier__epochs': [20],
    'kerasclassifier__learning_rate': [0.001],
    'kerasclassifier__activation_hidden': ['exponential'],
    'kerasclassifier__rate': [0.1, 0.2, 0.3, 0.4]
    
}

Cross Validation Score: 0.7985233565242085

Best Parameters: {'kerasclassifier__activation_hidden': 'exponential', 'kerasclassifier__batch_size': 80, 'kerasclassifier__epochs': 20, 'kerasclassifier__learning_rate': 0.001, 'kerasclassifier__rate': 0.1}

7: param_grid = {

    'kerasclassifier__batch_size': [80],
    'kerasclassifier__epochs': [20],
    'kerasclassifier__learning_rate': [0.001],
    'kerasclassifier__activation_hidden': ['exponential'],
    'kerasclassifier__rate': [0.1],
    'kerasclassifier__units': [12, 16, 32, 64]
    
}

Cross Validation Score: 0.802072980264092

Best Parameters: {'kerasclassifier__activation_hidden': 'exponential', 'kerasclassifier__batch_size': 80, 'kerasclassifier__epochs': 20, 'kerasclassifier__learning_rate': 0.001, 'kerasclassifier__rate': 0.1, 'kerasclassifier__units': 64}

8: param_grid = {

    'kerasclassifier__batch_size': [80],
    'kerasclassifier__epochs': [20, 50, 100, 200],
    'kerasclassifier__learning_rate': [0.001],
    'kerasclassifier__activation_hidden': ['exponential'],
    'kerasclassifier__rate': [0.1],
    'kerasclassifier__units': [64]
}

Cross Validation Score: 0.80335084481045

Best Parameters: {'kerasclassifier__activation_hidden': 'exponential', 'kerasclassifier__batch_size': 80, 'kerasclassifier__epochs': 50, 'kerasclassifier__learning_rate': 0.001, 'kerasclassifier__rate': 0.1, 'kerasclassifier__units': 64}

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?